In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [3]:
data = load_iris()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target)

In [4]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

In [5]:
# ===== Решающее дерево (упрощённое) =====
class SimpleDecisionTree:
    def __init__(self, max_depth=5, min_samples_split=2, n_features=None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.n_features = n_features
        self.tree = None
        
    def fit(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        # stopping criteria
        if (depth >= self.max_depth
            or n_labels == 1
            or n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return leaf_value

        # выбираем случайные признаки
        feat_idxs = np.random.choice(n_features, self.n_features, replace=False)

        # ищем лучший сплит
        best_feat, best_thresh = self._best_split(X, y, feat_idxs)

        # рекурсивное построение
        left_idxs, right_idxs = self._split(X[:, best_feat], best_thresh)

        left = self.fit(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self.fit(X[right_idxs, :], y[right_idxs], depth + 1)

        self.tree = (best_feat, best_thresh, left, right)
        return self.tree

    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_thresh = None, None

        for feat in feat_idxs:
            thresholds = np.unique(X[:, feat])
            for t in thresholds:
                gain = self._information_gain(y, X[:, feat], t)
                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat
                    split_thresh = t

        return split_idx, split_thresh

    def _information_gain(self, y, X_column, split_thresh):
        parent_entropy = self._entropy(y)

        left_idxs, right_idxs = self._split(X_column, split_thresh)
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_l/n)*e_l + (n_r/n)*e_r

        return parent_entropy - child_entropy

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _entropy(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p*np.log2(p) for p in ps if p > 0])

    def _most_common_label(self, y):
        return np.bincount(y).argmax()

    def predict(self, X):
        preds = [self._traverse(x, self.tree) for x in X]
        return np.array(preds)

    def _traverse(self, x, tree):
        if not isinstance(tree, tuple):
            return tree  # leaf

        feat, thresh, left, right = tree
        if x[feat] <= thresh:
            return self._traverse(x, left)
        return self._traverse(x, right)


# ===== Случайный лес =====
class SimpleRandomForest:
    def __init__(self, n_trees=10, max_depth=5, min_samples_split=2):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.trees = []

    def fit(self, X, y):
        n_samples, n_features = X.shape
        n_feat_subsample = int(np.sqrt(n_features))

        self.trees = []
        for _ in range(self.n_trees):
            tree = SimpleDecisionTree(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                n_features=n_feat_subsample
            )

            # bootstrap sample
            idxs = np.random.choice(n_samples, n_samples, replace=True)
            X_boot, y_boot = X[idxs], y[idxs]

            tree.fit(X_boot, y_boot)
            self.trees.append(tree)

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        # majority vote
        return np.apply_along_axis(
            lambda x: np.bincount(x).argmax(), axis=0, arr=tree_preds
        )

In [8]:
import numpy as np

# если уже numpy — оставим, если DataFrame — конвертим
X_train_np = np.array(X_train)
X_test_np = np.array(X_test)
y_train_np = np.array(y_train)
y_test_np = np.array(y_test)

forest.fit(X_train_np, y_train_np)
y_pred_custom = forest.predict(X_test_np)


In [9]:
print("Accuracy:", accuracy_score(y_test, y_pred_custom))
print("Precision:", precision_score(y_test, y_pred_custom, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_custom, average='weighted'))
print("F1:", f1_score(y_test, y_pred_custom, average='weighted'))

Accuracy: 0.9666666666666667
Precision: 0.9696969696969696
Recall: 0.9666666666666667
F1: 0.9665831244778613


In [10]:
# Сравнение со sklearn RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=10, max_depth=7, random_state=42)
rf.fit(X_train, y_train)

y_pred_sklearn = rf.predict(X_test)

print("\n=== SKLEARN RANDOM FOREST ===")
print("Accuracy:", accuracy_score(y_test, y_pred_sklearn))
print("Precision:", precision_score(y_test, y_pred_sklearn, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_sklearn, average='weighted'))
print("F1:", f1_score(y_test, y_pred_sklearn, average='weighted'))



=== SKLEARN RANDOM FOREST ===
Accuracy: 0.9666666666666667
Precision: 0.9696969696969696
Recall: 0.9666666666666667
F1: 0.9665831244778613
